# 🎓 GNN Project - Real AMiner Author Network
## Complete Self-Contained Implementation

**Dataset:** AMiner Author Network (Real Data)
**Tasks:**
- 🎯 Node Classification: Research field prediction
- 🔗 Link Prediction: Author collaboration prediction

**Models:** GAT, GCN, GraphSAGE

In [ ]:
# Install packages (uncomment to install)
# !pip install torch torchvision
# !pip install torch-geometric
# !pip install matplotlib pandas numpy scikit-learn requests

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
from torch_geometric.data import Data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from pathlib import Path
import json
import requests
import gzip
import tarfile
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)
np.random.seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cpu


## 📥 Download & Process Real AMiner Dataset

In [2]:
def download_aminer():
    """Use existing AMiner-Author.txt dataset"""
    
    data_dir = Path('data/aminer')
    author_file = data_dir / 'AMiner-Author.txt'
    
    if author_file.exists():
        print('✅ AMiner-Author.txt found')
        return author_file
    else:
        print('❌ AMiner-Author.txt not found')
        print('💡 Creating synthetic dataset...')
        return None

aminer_file = download_aminer()

✅ AMiner-Author.txt found


In [3]:
def parse_aminer(author_file, max_authors=10000):
    """Parse AMiner Author data and create author collaboration graph"""
    
    if author_file is None:
        print('Using synthetic data fallback...')
        num_nodes = max_authors
        X = torch.randn(num_nodes, 128)
        X = F.normalize(X, p=2, dim=1)
        y = torch.randint(0, 40, (num_nodes,))
        edges = []
        for i in range(num_nodes):
            for _ in range(np.random.randint(3, 10)):
                j = np.random.randint(0, num_nodes)
                if i != j:
                    edges.append([i, j])
        edge_index = torch.tensor(edges, dtype=torch.long).t()
        return X, edge_index, y, num_nodes, 40
    
    print(f'📖 Parsing AMiner Author data (limiting to {max_authors} authors)...')
    
    authors = []
    author_topics = []
    all_topics = set()
    
    with open(author_file, 'r', encoding='utf-8', errors='ignore') as f:
        current_author = {}
        count = 0
        
        for line in f:
            line = line.strip()
            
            if line.startswith('#index'):
                if current_author and 'topics' in current_author:
                    authors.append(current_author)
                    author_topics.append(current_author['topics'])
                    count += 1
                    if count >= max_authors:
                        break
                    if count % 1000 == 0:
                        print(f'\rParsed {count} authors...', end='')
                current_author = {'index': int(line.split()[1])}
            elif line.startswith('#n'):
                current_author['name'] = line[3:].strip()
            elif line.startswith('#pc'):
                current_author['paper_count'] = int(line.split()[1]) if len(line.split()) > 1 else 0
            elif line.startswith('#cn'):
                current_author['citation_count'] = int(line.split()[1]) if len(line.split()) > 1 else 0
            elif line.startswith('#hi'):
                current_author['h_index'] = int(line.split()[1]) if len(line.split()) > 1 else 0
            elif line.startswith('#t'):
                topics = [t.strip() for t in line[3:].strip().split(';') if t.strip()]
                current_author['topics'] = topics
                all_topics.update(topics)
    
    print(f'\n✅ Parsed {len(authors)} authors with {len(all_topics)} unique topics')
    
    topic_counter = Counter([t for topics in author_topics for t in topics])
    top_topics = [topic for topic, _ in topic_counter.most_common(500)]
    topic_to_idx = {topic: idx for idx, topic in enumerate(top_topics)}
    
    print(f'📊 Using top {len(top_topics)} topics for features')
    
    num_authors = len(authors)
    X = torch.zeros(num_authors, len(top_topics) + 3)
    
    for i, author in enumerate(authors):
        for topic in author['topics']:
            if topic in topic_to_idx:
                X[i, topic_to_idx[topic]] = 1
        X[i, -3] = min(author.get('paper_count', 0) / 100.0, 1.0)
        X[i, -2] = min(author.get('citation_count', 0) / 1000.0, 1.0)
        X[i, -1] = min(author.get('h_index', 0) / 50.0, 1.0)
    
    X = F.normalize(X, p=2, dim=1)
    
    print('🔗 Creating edges based on shared research topics...')
    edges = []
    topic_authors = defaultdict(list)
    for i, topics in enumerate(author_topics):
        for topic in topics:
            if topic in topic_to_idx:
                topic_authors[topic].append(i)
    
    edge_set = set()
    for topic, author_list in topic_authors.items():
        for i in range(len(author_list)):
            for j in range(i + 1, min(i + 10, len(author_list))):
                src, dst = author_list[i], author_list[j]
                if (src, dst) not in edge_set and (dst, src) not in edge_set:
                    edges.append([src, dst])
                    edges.append([dst, src])
                    edge_set.add((src, dst))
    
    edge_index = torch.tensor(edges, dtype=torch.long).t() if edges else torch.zeros((2, 0), dtype=torch.long)
    
    print('🏷️  Creating research field labels...')
    field_keywords = {
        'machine learning': 0, 'deep learning': 0, 'neural network': 0,
        'computer vision': 1, 'image processing': 1, 'object detection': 1,
        'natural language': 2, 'text mining': 2, 'language model': 2,
        'database': 3, 'data mining': 3, 'big data': 3,
        'network': 4, 'wireless': 4, 'communication': 4,
        'security': 5, 'encryption': 5, 'cryptography': 5,
        'algorithm': 6, 'optimization': 6, 'complexity': 6,
        'software': 7, 'programming': 7, 'development': 7,
        'system': 8, 'operating system': 8, 'distributed': 8,
        'robotics': 9, 'control': 9, 'automation': 9,
    }
    
    y = torch.zeros(num_authors, dtype=torch.long)
    for i, topics in enumerate(author_topics):
        for topic in topics[:3]:
            topic_lower = topic.lower()
            for keyword, label in field_keywords.items():
                if keyword in topic_lower:
                    y[i] = label
                    break
            if y[i] != 0:
                break
    
    num_classes = len(set(field_keywords.values()))
    
    print(f'📊 Created graph: {num_authors} authors, {edge_index.shape[1]} edges, {num_classes} fields')
    
    return X, edge_index, y, num_authors, num_classes

X, edge_index, y, num_authors, num_classes = parse_aminer(aminer_file, max_authors=10000)

📖 Parsing AMiner Author data (limiting to 10000 authors)...
Parsed 9000 authors...
✅ Parsed 10000 authors with 68682 unique topics
📊 Using top 500 topics for features
🔗 Creating edges based on shared research topics...
🏷️  Creating research field labels...
📊 Created graph: 10000 authors, 67018 edges, 10 fields


In [4]:
train_mask = torch.zeros(num_authors, dtype=torch.bool)
val_mask = torch.zeros(num_authors, dtype=torch.bool)
test_mask = torch.zeros(num_authors, dtype=torch.bool)

indices = torch.randperm(num_authors)
train_mask[indices[:int(0.6 * num_authors)]] = True
val_mask[indices[int(0.6 * num_authors):int(0.8 * num_authors)]] = True
test_mask[indices[int(0.8 * num_authors):]] = True

data = Data(x=X, edge_index=edge_index, y=y)
data.train_mask = train_mask
data.val_mask = val_mask
data.test_mask = test_mask

print(f'\n📊 Dataset Statistics:')
print(f'Authors: {data.num_nodes:,}')
print(f'Connections: {data.num_edges:,}')
print(f'Features: {data.num_features}')
print(f'Research Fields: {num_classes}')
print(f'Train: {train_mask.sum():,} | Val: {val_mask.sum():,} | Test: {test_mask.sum():,}')


📊 Dataset Statistics:
Authors: 10,000
Connections: 67,018
Features: 503
Research Fields: 10
Train: 6,000 | Val: 2,000 | Test: 2,000


## 🏗️ Model Definitions

In [5]:
class GAT(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=4, dropout=0.3):
        super().__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1, concat=False)
        self.dropout = dropout
    
    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        return self.conv2(x, edge_index)

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout
    
    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        return self.conv2(x, edge_index)

class GraphSAGE(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.3):
        super().__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)
        self.dropout = dropout
    
    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)
        return self.conv2(x, edge_index)

print('✅ Models defined')

✅ Models defined


## 🎯 Task 1: Node Classification (Research Field Prediction)

In [6]:
def train_node_classification(model, data, epochs=100, lr=0.01):
    model = model.to(device)
    data = data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    history = {'train_loss': [], 'val_acc': []}
    best_val_acc = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            out = model(data.x, data.edge_index)
            pred = out.argmax(dim=1)
            val_acc = (pred[data.val_mask] == data.y[data.val_mask]).float().mean()
            history['train_loss'].append(loss.item())
            history['val_acc'].append(val_acc.item())
            if val_acc > best_val_acc:
                best_val_acc = val_acc.item()
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}: Loss={loss.item():.4f}, Val Acc={val_acc:.4f}')
    
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        test_acc = (pred[data.test_mask] == data.y[data.test_mask]).float().mean()
    
    print(f'Final: Test Acc={test_acc:.4f}\n')
    return history, test_acc.item()

In [7]:
print('Training GAT...')
gat = GAT(data.num_features, 128, num_classes)
gat_hist, gat_acc = train_node_classification(gat, data, epochs=100)

Training GAT...
Epoch 20: Loss=0.9758, Val Acc=0.6940
Epoch 40: Loss=0.8855, Val Acc=0.6950
Epoch 60: Loss=0.8574, Val Acc=0.6925
Epoch 80: Loss=0.8395, Val Acc=0.6910
Epoch 100: Loss=0.8296, Val Acc=0.6890
Final: Test Acc=0.6935



In [8]:
print('Training GCN...')
gcn = GCN(data.num_features, 128, num_classes)
gcn_hist, gcn_acc = train_node_classification(gcn, data, epochs=100)

Training GCN...
Epoch 20: Loss=1.1252, Val Acc=0.6840
Epoch 40: Loss=0.9726, Val Acc=0.6945
Epoch 60: Loss=0.9226, Val Acc=0.6895
Epoch 80: Loss=0.8940, Val Acc=0.6890
Epoch 100: Loss=0.8761, Val Acc=0.6880
Final: Test Acc=0.6900



In [9]:
print('Training GraphSAGE...')
sage = GraphSAGE(data.num_features, 128, num_classes)
sage_hist, sage_acc = train_node_classification(sage, data, epochs=100)

Training GraphSAGE...
Epoch 20: Loss=1.0479, Val Acc=0.6925
Epoch 40: Loss=0.8892, Val Acc=0.6860
Epoch 60: Loss=0.8455, Val Acc=0.6865
Epoch 80: Loss=0.8264, Val Acc=0.6865
Epoch 100: Loss=0.8154, Val Acc=0.6895
Final: Test Acc=0.6895



In [10]:
results = pd.DataFrame({'Model': ['GAT', 'GCN', 'GraphSAGE'], 'Test Accuracy': [gat_acc, gcn_acc, sage_acc]})
print('='*50)
print('RESEARCH FIELD CLASSIFICATION RESULTS')
print('='*50)
print(results.to_string(index=False))

RESEARCH FIELD CLASSIFICATION RESULTS
    Model  Test Accuracy
      GAT         0.6935
      GCN         0.6900
GraphSAGE         0.6895


## 🔗 Task 2: Link Prediction (Author Collaboration Prediction)

In [11]:
# Create edge splits
num_edges = data.edge_index.shape[1]
perm = torch.randperm(num_edges)

num_val = int(0.1 * num_edges)
num_test = int(0.1 * num_edges)

train_edges = data.edge_index[:, perm[num_val + num_test:]]
val_pos = data.edge_index[:, perm[:num_val]]
test_pos = data.edge_index[:, perm[num_val:num_val + num_test]]

def sample_negative_edges(num_samples, existing_edges):
    edge_set = set(map(tuple, existing_edges.t().tolist()))
    neg_edges = []
    while len(neg_edges) < num_samples:
        src = torch.randint(0, data.num_nodes, (1,)).item()
        dst = torch.randint(0, data.num_nodes, (1,)).item()
        if src != dst and (src, dst) not in edge_set:
            neg_edges.append([src, dst])
    return torch.tensor(neg_edges, dtype=torch.long).t()

val_neg = sample_negative_edges(num_val, data.edge_index)
test_neg = sample_negative_edges(num_test, data.edge_index)

print(f'Edge splits created:')
print(f'Train: {train_edges.shape[1]:,} | Val: {val_pos.shape[1]:,} | Test: {test_pos.shape[1]:,}')

Edge splits created:
Train: 53,616 | Val: 6,701 | Test: 6,701


In [12]:
def train_link_prediction(model, data, train_edges, val_pos, val_neg, test_pos, test_neg, epochs=50, lr=0.01):
    model = model.to(device)
    data_train = Data(x=data.x.to(device), edge_index=train_edges.to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    history = {'train_loss': [], 'val_auc': []}
    best_val_auc = 0
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        z = model(data_train.x, data_train.edge_index)
        
        num_samples = min(5000, train_edges.shape[1])
        pos_idx = torch.randperm(train_edges.shape[1])[:num_samples]
        pos_edges = train_edges[:, pos_idx].to(device)
        neg_edges = sample_negative_edges(num_samples, train_edges).to(device)
        
        pos_score = (z[pos_edges[0]] * z[pos_edges[1]]).sum(dim=1)
        neg_score = (z[neg_edges[0]] * z[neg_edges[1]]).sum(dim=1)
        
        loss = -torch.log(torch.sigmoid(pos_score) + 1e-15).mean() - torch.log(1 - torch.sigmoid(neg_score) + 1e-15).mean()
        loss.backward()
        optimizer.step()
        
        history['train_loss'].append(loss.item())
        
        if (epoch + 1) % 10 == 0:
            model.eval()
            with torch.no_grad():
                z = model(data_train.x, data_train.edge_index)
                pos_score = (z[val_pos.to(device)[0]] * z[val_pos.to(device)[1]]).sum(dim=1)
                neg_score = (z[val_neg.to(device)[0]] * z[val_neg.to(device)[1]]).sum(dim=1)
                scores = torch.cat([pos_score, neg_score]).cpu().numpy()
                labels = np.concatenate([np.ones(len(pos_score)), np.zeros(len(neg_score))])
                val_auc = roc_auc_score(labels, scores)
                history['val_auc'].append(val_auc)
                if val_auc > best_val_auc:
                    best_val_auc = val_auc
                print(f'Epoch {epoch+1}: Loss={loss.item():.4f}, Val AUC={val_auc:.4f}')
    
    model.eval()
    with torch.no_grad():
        z = model(data_train.x, data_train.edge_index)
        pos_score = (z[test_pos.to(device)[0]] * z[test_pos.to(device)[1]]).sum(dim=1)
        neg_score = (z[test_neg.to(device)[0]] * z[test_neg.to(device)[1]]).sum(dim=1)
        scores = torch.cat([pos_score, neg_score]).cpu().numpy()
        labels = np.concatenate([np.ones(len(pos_score)), np.zeros(len(neg_score))])
        test_auc = roc_auc_score(labels, scores)
    
    print(f'Final: Val AUC={best_val_auc:.4f}, Test AUC={test_auc:.4f}\n')
    return history, test_auc

In [13]:
print('Training GAT for Link Prediction...')
gat_link = GAT(data.num_features, 128, 64)
gat_link_hist, gat_test_auc = train_link_prediction(gat_link, data, train_edges, val_pos, val_neg, test_pos, test_neg, epochs=50)

Training GAT for Link Prediction...
Epoch 10: Loss=0.8345, Val AUC=0.9918
Epoch 20: Loss=0.7917, Val AUC=0.9940
Epoch 30: Loss=0.7505, Val AUC=0.9965
Epoch 40: Loss=0.7484, Val AUC=0.9973
Epoch 50: Loss=0.7433, Val AUC=0.9976
Final: Val AUC=0.9976, Test AUC=0.9977



In [14]:
print('Training GCN for Link Prediction...')
gcn_link = GCN(data.num_features, 128, 64)
gcn_link_hist, gcn_test_auc = train_link_prediction(gcn_link, data, train_edges, val_pos, val_neg, test_pos, test_neg, epochs=50)

Training GCN for Link Prediction...
Epoch 10: Loss=0.9597, Val AUC=0.9551
Epoch 20: Loss=0.8999, Val AUC=0.9686
Epoch 30: Loss=0.8253, Val AUC=0.9885
Epoch 40: Loss=0.7770, Val AUC=0.9927
Epoch 50: Loss=0.7682, Val AUC=0.9958
Final: Val AUC=0.9958, Test AUC=0.9966



In [15]:
print('Training GraphSAGE for Link Prediction...')
sage_link = GraphSAGE(data.num_features, 128, 64)
sage_link_hist, sage_test_auc = train_link_prediction(sage_link, data, train_edges, val_pos, val_neg, test_pos, test_neg, epochs=50)

Training GraphSAGE for Link Prediction...
Epoch 10: Loss=0.9955, Val AUC=0.9440
Epoch 20: Loss=0.9554, Val AUC=0.9527
Epoch 30: Loss=0.8983, Val AUC=0.9779
Epoch 40: Loss=0.8221, Val AUC=0.9872
Epoch 50: Loss=0.8025, Val AUC=0.9939
Final: Val AUC=0.9939, Test AUC=0.9942



In [16]:
results_link = pd.DataFrame({'Model': ['GAT', 'GCN', 'GraphSAGE'], 'Test AUC': [gat_test_auc, gcn_test_auc, sage_test_auc]})
print('='*50)
print('LINK PREDICTION RESULTS')
print('='*50)
print(results_link.to_string(index=False))

LINK PREDICTION RESULTS
    Model  Test AUC
      GAT  0.997734
      GCN  0.996558
GraphSAGE  0.994213


## 💾 Save Models

In [17]:
Path('saved_models').mkdir(exist_ok=True)
torch.save({
    'gat_node': gat.state_dict(),
    'gcn_node': gcn.state_dict(),
    'sage_node': sage.state_dict(),
    'gat_link': gat_link.state_dict(),
    'gcn_link': gcn_link.state_dict(),
    'sage_link': sage_link.state_dict(),
    'results_node': results.to_dict(),
    'results_link': results_link.to_dict()
}, 'saved_models/aminer_models.pt')
print('✅ All models saved!')

✅ All models saved!


## ✅ Complete!

You've successfully trained GNN models on **real AMiner author network data**!

**What the models learned:**
- 🎯 Predict research fields based on author topics and collaboration patterns
- 🔗 Predict potential author collaborations based on shared research interests

In [18]:
print('\n' + '='*60)
print('FINAL SUMMARY - AMiner Author Network Analysis')
print('='*60)
print('\n🎯 RESEARCH FIELD CLASSIFICATION')
print('-' * 60)
print(results.to_string(index=False))
print('\n🔗 AUTHOR COLLABORATION PREDICTION')
print('-' * 60)
print(results_link.to_string(index=False))
print('\n' + '='*60)
print('✅ All tasks completed successfully!')
print('='*60)


FINAL SUMMARY - AMiner Author Network Analysis

🎯 RESEARCH FIELD CLASSIFICATION
------------------------------------------------------------
    Model  Test Accuracy
      GAT         0.6935
      GCN         0.6900
GraphSAGE         0.6895

🔗 AUTHOR COLLABORATION PREDICTION
------------------------------------------------------------
    Model  Test AUC
      GAT  0.997734
      GCN  0.996558
GraphSAGE  0.994213

✅ All tasks completed successfully!
